only difference here is the database i'm using 
paladins1.db has the pro games on it

In [ ]:
import pandas as pd
import sqlite3 as sq
from sklearn.model_selection import train_test_split


conn = sq.connect("../paladins1.db")

df = pd.read_sql_query("SELECT * FROM match", conn)

df["result"] = df["result"].apply(lambda x: 0 if x == 'L' else 1)

df = pd.get_dummies(df, columns=["champion"], dtype=int)



Empty DataFrame
Columns: [username, champ_level, k, d, a, credits, cpm, result, champion, damage, weapon, healing, self_heal, taken, shielding, map, match_id]
Index: []


In [ ]:
champion_cols = [col for col in df.columns if col.startswith("champion_")]
agg_dict = {col: 'sum' for col in champion_cols}
agg_dict.update({"result" : "first", "match_id" : "first"})

df = df.groupby(["match_id", "result"], as_index=False).agg(agg_dict).reset_index()

X = df[champion_cols]
y = df["result"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import layers

model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_dim=X_train.shape[1]),
    layers.Dropout(0.3),                                            
    layers.Dense(32, activation='relu'),                            
    layers.Dense(1, activation='sigmoid')                            
])

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

history = model.fit(X_train, y_train, 
                    epochs=50, 
                    batch_size=32, 
                    validation_split=0.2, 
                    verbose=1)

In [ ]:

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

y_pred_proba = model.predict(X_test)
print(f"Predict accuracy: {y_pred_proba}")

loss = history.history["loss"]
plt.plot(np.arange(1,len(loss)+1),loss, label="loss")
val_loss = history.history["val_loss"]
plt.plot(np.arange(1,len(val_loss)+1),val_loss, label="val_loss")
plt.legend()